In [1]:
import string

def count_letters(name):
    """
    Generates a 27-element list for a given name.

    The first 26 elements contain the count of each letter of the alphabet.
    The 27th element is 1 if the name ends with 'i' or 'a', otherwise 0.

    Args:
        name (str): The input name.

    Returns:
        list: A list of 27 integers.
    """
    # Initialize the list with 27 zeros
    output_list = [0] * 27

    # Normalize the name to lowercase for case-insensitive counting
    normalized_name = name.lower()

    # Count the frequency of each letter (a-z) for the first 26 elements
    for char in normalized_name:
        if 'a' <= char <= 'z':
            # Calculate the position in the list (0 for 'a', 1 for 'b', etc.)
            position = ord(char) - ord('a')
            output_list[position] += 1

    # Set the 27th element based on the last letter
    # Use [-1] to get the last character of the string
    if normalized_name:
        last_letter = normalized_name[-1]
        if last_letter in ('i', 'a'):
            output_list[26] = 1
        else:
            output_list[26] = 0
    return output_list

In [2]:
def nameEmbedding(name,intGender):
  letter_counts = count_letters(name)
  letter_counts.append(intGender)
  #print(letter_counts
  return (letter_counts)



In [3]:
!pip install Faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.3 MB/s eta 0:00:00


In [4]:
from faker import Faker
import pandas as pd

    # Initialize Faker with the Indian English locale (en_IN)
fake = Faker('en_IN')



lstnameEmbedding=[]

    # Generate multiple Indian male names

for _ in range(800):
        lstnameEmbedding.append(nameEmbedding(fake.first_name_male(),0.1))
#lstnameEmbedding.append(nameEmbedding('Kamlesh',0.9))



    # Generate multiple Indian female names

for _ in range(800):
        lstnameEmbedding.append(nameEmbedding(fake.first_name_female(),0.9))
#lstnameEmbedding.append(nameEmbedding('Sangeetha',0.1))
#lstnameEmbedding.append(nameEmbedding('Mounika',0.1))
df = pd.DataFrame(lstnameEmbedding)
df.to_csv('nameEmbedding.csv', index=False)

In [5]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# load the dataset, split into input (X) and output (y) variables
dataset = np.genfromtxt('/content/nameEmbedding.csv', delimiter=',',skip_header=1)
X = dataset[:,0:27]
y = dataset[:,27]

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

# define the model
model = nn.Sequential(
    nn.Linear(27, 12),
    nn.ReLU(),
    nn.Linear(12, 8),
    nn.ReLU(),
    nn.Linear(8, 1),
    nn.Sigmoid()
)
print(model)

# train the model
loss_fn   = nn.BCELoss()  # binary cross entropy
optimizer = optim.Adam(model.parameters(), lr=0.0001)

n_epochs = 100
batch_size = 10

for epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
        Xbatch = X[i:i+batch_size]
        y_pred = model(Xbatch)
        ybatch = y[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    #print(f'Finished epoch {epoch}, latest loss {loss}')
print(f'Finished epoch {epoch}, latest loss {loss}')
# compute accuracy (no_grad is optional)


Sequential(
  (0): Linear(in_features=27, out_features=12, bias=True)
  (1): ReLU()
  (2): Linear(in_features=12, out_features=8, bias=True)
  (3): ReLU()
  (4): Linear(in_features=8, out_features=1, bias=True)
  (5): Sigmoid()
)
Finished epoch 99, latest loss 0.34543561935424805


In [6]:
def predictGenderFromName(strName):
  y_pred=model(torch.tensor(count_letters(strName),dtype=torch.float32))
  print(strName,':',y_pred)
  if y_pred >0.5:
      print('Female')
  else:
      print('Male')


In [7]:
predictGenderFromName('Anand')
predictGenderFromName('Vidhya')
predictGenderFromName('Anandhi')
predictGenderFromName('Ramesh')
predictGenderFromName('Shankar')
predictGenderFromName('Shankari')


Anand : tensor([0.1286], grad_fn=<SigmoidBackward0>)
Male
Vidhya : tensor([0.9064], grad_fn=<SigmoidBackward0>)
Female
Anandhi : tensor([0.9013], grad_fn=<SigmoidBackward0>)
Female
Ramesh : tensor([0.2158], grad_fn=<SigmoidBackward0>)
Male
Shankar : tensor([0.0872], grad_fn=<SigmoidBackward0>)
Male
Shankari : tensor([0.8254], grad_fn=<SigmoidBackward0>)
Female


In [8]:

#wrong predictions increase epoch to 1000 train the model again and verify
print('if Wrong , Increase epoch to 1000 and see results')
predictGenderFromName('Shreyas')
predictGenderFromName('Shreya')
predictGenderFromName('ArulMozhi')
predictGenderFromName('Jeyasree')
predictGenderFromName('Jasmine')
predictGenderFromName('Hari')

if Wrong , Increase epoch to 1000 and see results
Shreyas : tensor([0.0668], grad_fn=<SigmoidBackward0>)
Male
Shreya : tensor([0.7845], grad_fn=<SigmoidBackward0>)
Female
ArulMozhi : tensor([0.9326], grad_fn=<SigmoidBackward0>)
Female
Jeyasree : tensor([0.0380], grad_fn=<SigmoidBackward0>)
Male
Jasmine : tensor([0.1551], grad_fn=<SigmoidBackward0>)
Male
Hari : tensor([0.8719], grad_fn=<SigmoidBackward0>)
Female
